# Setup

Fall [these instructions](https://docs.astral.sh/uv/getting-started/installation/) to install `uv`. Then run the following command to set up your environment:

```bash
uv sync --quiet
```

In [1]:
# Imports
from robora import Workflow, SQLiteStorageProvider
from robora.sonar_query import SonarQueryHandler
from robora.mock_query import MockQueryHandler
from library.organization_question import OrganizationModel, get_question_set
from data import COUNTRIES, DOMAINS
import pandas as pd

In [ ]:
sqlstorage = SQLiteStorageProvider("organization.db")
workflow = Workflow(
    SonarQueryHandler(OrganizationModel),
    sqlstorage,
    workers = 4,
    )
question_set = get_question_set(
    domains=['Energy'],
    countries=COUNTRIES,
)
question_set.max_questions = 0  # No limit

Loaded 0 stored question hashes from organization.db


In [3]:
await workflow.ask_multiple(question_set, return_results=False)

ask_multiple: gathering answers
ask_multiple_stream: starting for 3 questions with 4 workers
Processing question 1/3: What is the top-level state Organ (i.e., ministry/department/agency) responsible for {domain} in {country}? - {'domain': 'ENERGY', 'country': 'ALBANIA'}
Processing question 2/3: What is the top-level state Organ (i.e., ministry/department/agency) responsible for {domain} in {country}? - {'domain': 'ENERGY', 'country': 'ALGERIA'}
Processing question 3/3: What is the top-level state Organ (i.e., ministry/department/agency) responsible for {domain} in {country}? - {'domain': 'ENERGY', 'country': 'AFGHANISTAN'}
Finished question 3/3
Finished question 3/3
Finished question 3/3
ask_multiple: completed, collected 0 answers


[]

In [4]:
answers = []
async for answer in workflow.dump_answers():
    answers.append(answer)
flattened = pd.concat([ans.flattened for ans in answers]).reset_index(drop=True)
flattened

,question,error,domain,country,organization_name,confidence,enriched_citations
0,"What is the top-level state Organ (i.e., minis...",None,ENERGY,ALGERIA,Ministry of Energy and Renewable Energy,ConfidenceLevel.MEDIUM,[{'url': 'https://en.wikipedia.org/wiki/Minist...
1,"What is the top-level state Organ (i.e., minis...",None,ENERGY,AFGHANISTAN,Ministry of Energy and Water,ConfidenceLevel.HIGH,[{'url': 'https://en.wikipedia.org/wiki/Minist...
2,"What is the top-level state Organ (i.e., minis...",None,ENERGY,ALBANIA,Ministry of Infrastructure and Energy (Albania),ConfidenceLevel.HIGH,[{'url': 'https://www.devex.com/organizations/...


In [5]:
wrangled = flattened.copy()


In [6]:
wrangled.enriched_citations

0    [{'url': 'https://en.wikipedia.org/wiki/Minist...
1    [{'url': 'https://en.wikipedia.org/wiki/Minist...
2    [{'url': 'https://www.devex.com/organizations/...
Name: enriched_citations, dtype: object

In [ ]:
wrangled.to_excel("organization_names.xlsx", index=False)